# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234573185315                   -0.50    7.0
  2   -7.249907956736       -1.81       -1.38    1.0
  3   -7.251127779567       -2.91       -1.97    2.0
  4   -7.251046246950   +   -4.09       -1.93    2.0
  5   -7.251329609871       -3.55       -2.67    1.0
  6   -7.251337978429       -5.08       -3.14    1.0
  7   -7.251338703392       -6.14       -3.58    2.0
  8   -7.251338788422       -7.07       -4.06    1.0
  9   -7.251338795932       -8.12       -4.31    3.0
 10   -7.251338798445       -8.60       -4.92    1.0
 11   -7.251338798686       -9.62       -5.50    2.0
 12   -7.251338798699      -10.86       -5.58    3.0
 13   -7.251338798702      -11.54       -5.81    1.0
 14   -7.251338798704      -11.66       -6.70    2.0
 15   -7.251338798705      -12.96       -6.96    3.0
 16   -7.251338798705      -14.57       -7.55    1.0
 17   -7.251338798705   +    -Inf       -8.11 

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05094706522808781
[ Info: Arnoldi iteration step 2: normres = 0.40826544388630803
[ Info: Arnoldi iteration step 3: normres = 0.8262754114729384
[ Info: Arnoldi iteration step 4: normres = 0.4476853107903967
[ Info: Arnoldi iteration step 5: normres = 0.565565422476882
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.17e-01, 8.31e-02, 5.05e-01, 2.10e-01, 1.22e-02)
[ Info: Arnoldi iteration step 6: normres = 0.28024874531456545
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.69e-02, 1.25e-01, 1.93e-01, 1.13e-01, 8.37e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08658788494163913
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.13e-03, 1.09e-02, 1.39e-02, 5.44e-02, 5.71e-02)
[ Info: Arnoldi iteration step 8: normres = 0.13079299691210497
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.48e-